# CALVADOS 3 Colab
Here is the colab version of [CALVADOS 3](https://www.biorxiv.org/content/10.1101/2024.02.03.578735v3), designed for a quick single-chain simulation for your protein. It can be used for both of intrinsically disordered proteins (IDPs) and multi-domain proteins (MDPs).

C$_\alpha$ and center-of-mass (COM) representation will be used for IDPs and MDPs, respectively.

To run simulations, one needs to provide `temperature`, `ionic strength`, `pH`, `sequence` and `domain boundary`. Detailed instructions can be found in block 2.

[Fan Cao, Sören von Bülow, Giulio Tesei, Kresten Lindorff-Larsen. A coarse-grained model for disordered and multi-domain proteins](https://www.biorxiv.org/content/10.1101/2024.02.03.578735v3)

In [ ]:
# @title 1.1 (Optional) Install [cg2all](https://www.sciencedirect.com/science/article/pii/S096921262300401X?via%3Dihub) for all-atom reconstruction
%%bash
pip install dgl -f https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html &> /dev/null
pip install -q git+http://github.com/huhlim/cg2all@cuda-12 &> /dev/null
pip install -q py3Dmol gdown mrcfile &> /dev/null

In [ ]:
# @title 1.2 Set the environment for simulation

# @markdown Attention! A prompt of "Your session crashed for an unknown reason" will appear after running this block. Please ignore it and move on.

# @markdown If you have many proteins to run, change the inputs in block 2, 3 and 4 for each protein as you need. You only need to run block 1 once.
import subprocess
import locale
locale.getpreferredencoding = lambda: "UTF-8"
subprocess.run('pip install MDAnalysis'.split())
subprocess.run('pip install mdtraj'.split())
subprocess.run('pip install -q condacolab'.split())
import condacolab
condacolab.install()
subprocess.run('mamba install openmm -c conda-forge --yes'.split())
subprocess.run('pip install wget'.split())
subprocess.run('pip install fastprogress'.split())
subprocess.run('mamba install pandas'.split())
subprocess.run('mamba install pydantic'.split())
# !pip install "dgl<=1.0"  # cpu for cg2all

In [ ]:
# @title 2. Configure your protein

# @markdown Please check `gpu` if you are using a GPU session.

import numpy as np
import os
import shutil
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

gpu = False  #@param {type:"boolean"}
example = False  #@param {type:"boolean"}
# @markdown If `example` is checked, all the inputs below in this block will be ignored. In this case, [THB_C2](https://www.biorxiv.org/content/10.1101/2024.02.03.578735v3) will serve as an example for tutorial with pre-defined settings.

isIDP = False #@param {type:"boolean"}
name = "THB_C2" #@param {type:"string"}
sequence = "GPGSEDVWEILRQAPPSEYERIAFQYGVTDLRGMLKRLKGMRRDEKKSTAFQKKLEPAYQVSKGHKIRLTVELADHDAEVKWLKNGQEIQMSGSKYIFESIGAKRTLTISQCSLADDAAYQCVVGGEKCSTELFVKE" #@param {type:"string"}
L = int(np.ceil((len(sequence) - 1) * 0.38 + 4))
if L > 900:  # nm
    raise Exception("Too large box! Floats might overflow in PDB file. You can try to modify the 'L' parameter in this block yourself.")
temperature = 295.15 #@param {type:"number"}
ionic_strength = 0.15 #@param {type:"number"}
pH = 6.5 #@param {type:"number"}
cutoff = 2.0  # nm
#@markdown <i>*Units: temperature [K], ionic strength [M]<i>

#@markdown <b>Define domain boundary -- ignore this if `isIDP` is toggled:

#@markdown Residue ranges are delimited by `-`, e.g. residues 1 to 200 are `1-200`;

#@markdown Separate domains are delimited by `,`, e.g. `1-200,201-400`;

#@markdown Discontinuous segments are delimited by `_`, e.g. `1-200,201-300_350-400`.
domain_boundary = "6-42,50-137" #@param {type:"string"}
input_pae = False
k_restraint = 700  # unit:KJ/(mol*nm^2)
eps_factor = 0.2
initial_type = "C3"
discard_first_nframes = 10

if example:
    name = "THB_C2"
    isIDP = False
    sequence = "GPGSEDVWEILRQAPPSEYERIAFQYGVTDLRGMLKRLKGMRRDEKKSTAFQKKLEPAYQVSKGHKIRLTVELADHDAEVKWLKNGQEIQMSGSKYIFESIGAKRTLTISQCSLADDAAYQCVVGGEKCSTELFVKE"
    L = int(np.ceil((len(sequence) - 1) * 0.38 + 4))
    temperature = 295.15
    ionic_strength = 0.15
    pH = 6.5
    domain_boundary = "6-42,50-137"

use_pdb = False if isIDP else True
use_hnetwork = False if isIDP else True
use_ssdomains = False if isIDP else True

if not isIDP:
    domain_resNum = []  # start from 1
    for domain in domain_boundary.split(","):
        domain = domain.split("_")
        if len(domain) == 1:  # no break point
          domain_resNum.append(list(range(int(domain[0].split('-')[0]), int(domain[0].split('-')[1]) + 1)))
        else:
          tmp_list = []
          for subdomain in domain:
              tmp_list += list(range(int(subdomain.split('-')[0]), int(subdomain.split('-')[1]) + 1))
          domain_resNum.append(tmp_list)
else:
    domain_boundary = None
    domain_resNum = None

print(f"{'IDP' if isIDP else 'MDP'} name: {name}")
print(f"sequence: {sequence}")
print(f"simulation box: [{L}, {L}, {L}] nm")
print(f"temperature: {temperature} K")
print(f"ionic strength: {ionic_strength} M")
print(f"pH: {pH}")
if not isIDP:
    print(f"There are {len(domain_resNum)} restrained domains (1-based): ")
    for d in domain_resNum:
        print(d)
print(f"{'GPU' if gpu else 'CPU'} will be used.")
if not os.path.isdir(f"{name}"):
    os.system(f"mkdir -p {name}")

In [ ]:
# @title 3. Upload a pdb file
# @markdown No need to upload any file when `example` in the previous block is checked.

# @markdown *Please make sure the uploaded PDB file starts `resSeq` from 1; Scripts won't check.


from google.colab import files
import wget

if not isIDP:
    if example:
        wget.download('https://github.com/KULL-Centre/_2024_Cao_CALVADOSCOM/raw/main/src/extract_relax/THB_C2_rank0_relax.pdb')
        os.system(f"mv THB_C2_rank0_relax.pdb {name}")
        path2pdb = f"{name}/THB_C2_rank0_relax.pdb"
    else:
        uploaded = files.upload()
        if len(list(uploaded.keys())) != 1:
          raise Exception("Please upload just one pdb file")
        for key in uploaded.keys():
            os.system(f"mv {key} {name}")
        path2pdb = f"{name}/{list(uploaded.keys())[0]}"
else:
    path2pdb = ''
if path2pdb == '':
    print("No need to upload pdb file.")
else:
    print(f"{path2pdb} successfully uploaded.")
wget.download('https://github.com/KULL-Centre/_2024_Cao_CALVADOSCOM/raw/main/src/residues_pub.csv')
os.system(f"mv residues_pub.csv {name}")

In [ ]:
# @title 4. Determine simulation time (unit: ns)

# @markdown If the value is `AUTO`, the simulated time will be determined by the autocorrelation analysis according to our [previous work](https://open-research-europe.ec.europa.eu/articles/2-94/v2)
import yaml
Simulation_time = "AUTO" #@param {type:"string"}
N_res = len(sequence)
if not isIDP:
  domain_len = 0
  for domain in domain_resNum:
    domain_len += len(domain)
  N_res -= domain_len

N_save = 3000 if N_res < 100 else int(np.ceil(3e-4 * N_res ** 2) * 1000)

if Simulation_time == "AUTO":
    N_steps = 210 * int(N_save)
    print(f'AUTO simulation length selected. Running for {N_steps*0.01/1000} ns, {N_steps} steps in total.')
else:
    N_steps = int(float(Simulation_time)*1000/0.01)
    print(f'Simulation length assigned. Running for {Simulation_time} ns, {N_steps} steps in total.')

nframes = N_steps // N_save
config_sim_data = dict(
temp=float(temperature), ionic=float(ionic_strength),
pH=float(pH), cutoff=cutoff,
L=L, wfreq=int(N_save), use_pdb=use_pdb,
path2pdb=path2pdb, use_hnetwork=use_hnetwork, domain_resNum=domain_resNum,
use_ssdomains=use_ssdomains, input_pae=input_pae,
k_restraint=k_restraint, record=name,
gpu_id=0, N_res=int(N_res),
CoarseGrained="COM", isIDP=isIDP,
eps_factor=eps_factor,
initial_type=initial_type, seq=list(sequence), steps=N_steps, gpu=gpu,
discard_first_nframes=discard_first_nframes,
nframes=nframes)
yaml.dump(config_sim_data, open(f'{name}/config_sim.yaml', 'w'))

In [ ]:
# @title 5. Run simulations
import time
from openmm import app
from simtk import openmm, unit
import pandas as pd
import MDAnalysis as mda
import mdtraj as md
from fastprogress import master_bar, progress_bar


def geometry_from_pdb(record, pdb, CoarseGrained="CA", ssdomains=None):
    """ positions in nm """
    u = mda.Universe(pdb)
    ini_CA = u.select_atoms("name CA")
    ini_CA.write(f"{record}/ini_CA.pdb")
    if CoarseGrained == "CA":
        cas = u.select_atoms('name CA')  # in-place
        center_of_mass = cas.center_of_mass()
        cas.translate(-center_of_mass)
        pos = cas.positions / 10.  # nm, shape: (n, 3)
    elif CoarseGrained == "COM":  # COM
        pos = []
        resSeqinDomain = []
        for ssdomain in ssdomains:
            resSeqinDomain += ssdomain
        for i in range(len(u.residues)):  # i starts from 0
            if i+1 in resSeqinDomain:  # COM
                pos.append(u.residues[i].atoms.center_of_mass())
            else:  # CA
                pos.append(np.squeeze(u.residues[i].atoms.select_atoms("name CA").positions))
        center_of_mass = u.atoms.center_of_mass()
        pos = (np.array(pos) - center_of_mass) / 10.  # nm

    return pos, center_of_mass/10.  # nm

def genParamsDH(df,record,prot,temp):
    kT = 8.3145*temp*1e-3
    fasta = prot.fasta.copy()
    r = df.copy()
    q = 1.0

    r.loc['H','q'] = q / ( 1 + 10**(prot.pH-6) )
    r.loc['X'] = r.loc[fasta[0]]
    r.loc['Z'] = r.loc[fasta[-1]]
    fasta[0] = 'X'
    fasta[-1] = 'Z'
    r.loc['X','q'] = r.loc[prot.fasta[0],'q'] + q
    r.loc['Z','q'] = r.loc[prot.fasta[-1],'q'] - q
    # Calculate the prefactor for the Yukawa potential
    fepsw = lambda T : 5321/T+233.76-0.9297*T+0.1417*1e-2*T*T-0.8292*1e-6*T**3
    epsw = fepsw(temp)
    lB = 1.6021766**2/(4*np.pi*8.854188*epsw)*6.022*1000/kT
    yukawa_eps = [r.loc[a].q*np.sqrt(lB*kT) for a in fasta]
    # Calculate the inverse of the Debye length
    yukawa_kappa = np.sqrt(8*np.pi*lB*prot.ionic*6.022/10)
    return yukawa_eps, yukawa_kappa

def genParamsLJ(df,record,prot):
    fasta = prot.fasta.copy()
    r = df.copy()
    r.loc['X'] = r.loc[fasta[0]]
    r.loc['Z'] = r.loc[fasta[-1]]
    r.loc['X','MW'] += 2
    r.loc['Z','MW'] += 16
    fasta[0] = 'X'
    fasta[-1] = 'Z'
    types = list(np.unique(fasta))
    MWs = [r.loc[a,'MW'] for a in types]
    lj_eps = prot.eps_factor*4.184
    return lj_eps, fasta, types, MWs

def load_parameters(record, initial_type, output=False):
    residues = pd.read_csv(f'{record}/residues_pub.csv').set_index('one', drop=False)
    if initial_type == "C1":
        residues["lambdas"] = residues["CALVADOS1"]
        residues.to_csv(f'{record}/residues_pub.csv')
    if initial_type == "C2":
        residues["lambdas"] = residues["CALVADOS2"]
        residues.to_csv(f'{record}/residues_pub.csv')
    if initial_type == "C3":
        residues["lambdas"] = residues["CALVADOS3"]
        residues.to_csv(f'{record}/residues_pub.csv')
    if output:
        print("Lambda used:\n", residues.lambdas)
    return residues

def p2c(r, phi):
    """
    polar to cartesian
    """
    return (r * np.cos(phi), r * np.sin(phi))

def xy_spiral_array(n, delta=0, arc=.38, separation=.7):
    """
    create points on an Archimedes' spiral
    with `arc` giving the length of arc between two points
    and `separation` giving the distance between consecutive
    turnings
    """
    r = arc
    b = separation / (2 * np.pi)
    phi = float(r) / b
    coords = []
    for i in range(n):
        coords.append(list(p2c(r, phi))+[0])
        phi += float(arc) / r
        r = b * phi
    return np.array(coords)+delta

def build_topology(fasta,n_chains=1):
    # build CG topology
    top = md.Topology()
    for i_chain in range(n_chains):
        chain = top.add_chain()
        for resname in fasta:
            residue = top.add_residue(resname, chain)
            top.add_atom(resname, element=md.element.carbon, residue=residue)
        for i in range(chain.n_atoms-1):
            top.add_bond(chain.atom(i),chain.atom(i+1))
    return top

def build_box(Lx,Ly,Lz):
    # set box vectors
    a = unit.Quantity(np.zeros([3]), unit.nanometers)
    a[0] = Lx * unit.nanometers
    b = unit.Quantity(np.zeros([3]), unit.nanometers)
    b[1] = Ly * unit.nanometers
    c = unit.Quantity(np.zeros([3]), unit.nanometers)
    c[2] = Lz * unit.nanometers
    return a, b, c

def add_particles(system, residues, prot, n_chains=1):
    for i_chain in range(n_chains):
        system.addParticle((residues.loc[prot.fasta[0]].MW+2)*unit.amu)
        for a in prot.fasta[1:-1]:
            system.addParticle(residues.loc[a].MW*unit.amu)
        system.addParticle((residues.loc[prot.fasta[-1]].MW+16)*unit.amu)
    return system

def euclidean(a_matrix, b_matrix):
    # Using matrix operation to calculate Euclidean distance
    """                     b
       [[2.23606798  1.          5.47722558]
    a   [7.07106781  4.69041576  3.        ]
        [12.20655562 9.8488578   6.4807407 ]]"""
    d1 = -2 * np.dot(a_matrix, b_matrix.T)
    d2 = np.sum(np.square(a_matrix), axis=1, keepdims=True)
    d3 = np.sum(np.square(b_matrix), axis=1)
    dist = np.sqrt(d1 + d2 + d3)
    return dist

def set_interactions(system, residues, prot, lj_eps, cutoff, yukawa_kappa, yukawa_eps, N, n_chains=1,
                     CoarseGrained="CA", dismatrix=None, isIDP=True, domain_resNum=None):
    hb = openmm.openmm.HarmonicBondForce()
    # interactions
    energy_expression = 'select(step(r-2^(1/6)*s),4*eps*l*((s/r)^12-(s/r)^6-shift),4*eps*((s/r)^12-(s/r)^6-l*shift)+eps*(1-l))'
    ah = openmm.openmm.CustomNonbondedForce(energy_expression + '; s=0.5*(s1+s2); l=0.5*(l1+l2); shift=(0.5*(s1+s2)/rc)^12-(0.5*(s1+s2)/rc)^6')
    ah.addGlobalParameter('eps', lj_eps * unit.kilojoules_per_mole)
    ah.addGlobalParameter('rc', float(cutoff) * unit.nanometer)
    ah.addPerParticleParameter('s')
    ah.addPerParticleParameter('l')
    print('rc', cutoff * unit.nanometer)

    yu = openmm.openmm.CustomNonbondedForce('q*(exp(-kappa*r)/r-shift); q=q1*q2')
    yu.addGlobalParameter('kappa', yukawa_kappa / unit.nanometer)
    yu.addGlobalParameter('shift', np.exp(-yukawa_kappa * 4.0) / 4.0 / unit.nanometer)
    yu.addPerParticleParameter('q')

    for j in range(n_chains):
        begin = j * N  # 0
        end = j * N + N  # n_residues
        for a, e in zip(prot.fasta, yukawa_eps):
            yu.addParticle([e * unit.nanometer * unit.kilojoules_per_mole])
            ah.addParticle([residues.loc[a].sigmas * unit.nanometer, residues.loc[a].lambdas * unit.dimensionless])

        for i in range(begin, end - 1):  # index starts from 0
            if CoarseGrained=="CA" or isIDP:
                hb.addBond(i, i + 1, 0.38 * unit.nanometer, 8033.0 * unit.kilojoules_per_mole / (unit.nanometer ** 2))
            else:  # COM or COE
                ssdomains = domain_resNum
                resSeqinDomain = []
                for ssdomain in ssdomains:
                    resSeqinDomain += ssdomain
                if i+1 in resSeqinDomain or i+1+1 in resSeqinDomain:
                    hb.addBond(i, i + 1, dismatrix[i][i+1] * unit.nanometer, 8033.0 * unit.kilojoules_per_mole / (unit.nanometer ** 2))
                else:
                    hb.addBond(i, i + 1, 0.38 * unit.nanometer, 8033.0 * unit.kilojoules_per_mole / (unit.nanometer ** 2))
            yu.addExclusion(i, i + 1)
            ah.addExclusion(i, i + 1)

    # All Forces in a single force group must use the same cutoff distance
    yu.setForceGroup(0)
    ah.setForceGroup(1)
    yu.setNonbondedMethod(openmm.openmm.CustomNonbondedForce.CutoffPeriodic)
    ah.setNonbondedMethod(openmm.openmm.CustomNonbondedForce.CutoffPeriodic)
    hb.setUsesPeriodicBoundaryConditions(True)
    yu.setCutoffDistance(4 * unit.nanometer)
    ah.setCutoffDistance(cutoff * unit.nanometer)
    return hb, yu, ah

def set_harmonic_network(N,dmap,pae_inv,yu,ah,ssdomains=None,cs_cutoff=0.9,k_restraint=700.):
    cs = openmm.openmm.HarmonicBondForce()
    for i in range(N-2):
        for j in range(i+2,N):
            if ssdomains != None:  # use fixed domain boundaries for network
                ss = False
                for ssdom in ssdomains:
                    if i+1 in ssdom and j+1 in ssdom:
                        ss = True
                if ss:  # both residues in structured domains
                    if dmap[i,j] < cs_cutoff:  # nm
                        cs.addBond(i, j, dmap[i, j] * unit.nanometer,
                                   k_restraint * unit.kilojoules_per_mole / (unit.nanometer ** 2))
                        yu.addExclusion(i, j)
                        ah.addExclusion(i, j)
            elif isinstance(pae_inv, np.ndarray):  # use alphafold PAE matrix for network
                k = k_restraint * pae_inv[i,j]**2
                if k > 0.0:
                    cs.addBond(i,j, dmap[i,j]*unit.nanometer,
                                k*unit.kilojoules_per_mole/(unit.nanometer**2))
                    yu.addExclusion(i, j)
                    ah.addExclusion(i, j)
            else:
                raise
    cs.setUsesPeriodicBoundaryConditions(True)
    return cs, yu, ah

def simulate(config, CUDA_VISIBLE_DEVICES=0):
    """ Simulate openMM Calvados

        * config is a dictionary """

    # parse config
    record, temp, ionic = config['record'], config['temp'], config['ionic']
    cutoff, steps, wfreq = config['cutoff'], config['steps'], config['wfreq']
    L = config['L']
    domain_resNum = config['domain_resNum']
    eps_factor = config["eps_factor"]
    pH = config["pH"]
    isIDP = config['isIDP']
    CoarseGrained = config['CoarseGrained']
    # gpu_id = config['gpu_id']  # mapping
    gpu = config['gpu']
    seq = config['seq']
    replica = 0
    use_pdb, path2pdb = config['use_pdb'], config['path2pdb']
    initial_type = config['initial_type']
    center_of_mass = np.array([0, 0, 0])
    if use_pdb:
        use_hnetwork = config['use_hnetwork']
        if use_hnetwork:
            k_restraint = config['k_restraint']
            use_ssdomains = config['use_ssdomains']
            if use_ssdomains:
                ssdomains = domain_resNum
                pae = None
            else:
                raise Exception("please make sure 'use_ssdomains' is True")
    else:
        path2pdb = ''
        use_hnetwork = False
        pae = None
        ssdomains = None
    # load residue parameters
    residues = load_parameters(record, initial_type, output=True)
    # build protein dataframe
    df = pd.DataFrame(columns=['pH', 'ionic', 'temp', 'eps_factor', 'fasta'], dtype=object)
    df.loc[record] = dict(pH=pH, ionic=ionic, temp=temp, eps_factor=eps_factor, fasta=seq)
    prot = df.loc[record]
    print("pH:", prot.pH)
    print("Ionic strength:", prot.ionic)
    print("Temperature:", prot.temp)
    # LJ and YU parameters
    lj_eps, fasta, types, MWs = genParamsLJ(residues, record, prot)
    # print("lj_eps:", lj_eps * unit.kilojoules_per_mole)
    yukawa_eps, yukawa_kappa = genParamsDH(residues, record, prot, temp)

    N = len(fasta)  # number of residues
    n_chains = 1
    Lz = L

    # get input geometry

    if use_pdb:
        print(f'Starting from pdb structure {path2pdb}')
        pos, center_of_mass = geometry_from_pdb(record, path2pdb, CoarseGrained=CoarseGrained, ssdomains=domain_resNum)
    else:
        spiral = True
        if spiral:
            pos = xy_spiral_array(N)
        else:
            pos = [[L / 2, L / 2, L / 2 + (i - N / 2.) * .38] for i in range(N)]
        pos = np.array(pos)

    top = build_topology(fasta, n_chains=n_chains)
    md.Trajectory(pos + center_of_mass, top, 0, [L, L, Lz], [90, 90, 90]).save_pdb(f'{record}/ini_beads.pdb', force_overwrite=True)
    pos = pos + np.array([L / 2, L / 2, L / 2])
    a = md.Trajectory(pos, top, 0, [L, L, Lz], [90, 90, 90])
    a.save_pdb(f'{record}/top_production.pdb', force_overwrite=True)
    # build openmm system
    system = openmm.System()

    # box
    a, b, c = build_box(L, L, Lz)
    system.setDefaultPeriodicBoxVectors(a, b, c)

    # load topology into system
    pdb = app.pdbfile.PDBFile(f'{record}/top_production.pdb')

    # print(pdb.topology)

    # particles and termini
    system = add_particles(system, residues, prot, n_chains=n_chains)
    dmap = euclidean(pos, pos)
    # interactions
    hb, yu, ah = set_interactions(system, residues, prot, lj_eps, cutoff, yukawa_kappa, yukawa_eps, N,
                                  n_chains=n_chains, CoarseGrained=CoarseGrained, dismatrix=dmap, isIDP=isIDP,
                                  domain_resNum=domain_resNum)
    system.addForce(hb)
    system.addForce(yu)
    system.addForce(ah)

    # print("use_hnetwork: ", use_hnetwork)
    # harmonic network (currently unavailable for slab sim)
    if use_hnetwork:
        cs, yu, ah = set_harmonic_network(N, dmap, pae, yu, ah, ssdomains=ssdomains, k_restraint=k_restraint)
        # print(f"k_restraint used: {k_restraint}")
        system.addForce(cs)

    open(f'{record}/system.xml', 'w').write(openmm.XmlSerializer.serialize(system))
    # use langevin integrator
    integrator = openmm.openmm.LangevinIntegrator(temp * unit.kelvin, 0.01 / unit.picosecond, 0.01 * unit.picosecond)
    # print(integrator.getFriction(), integrator.getTemperature())

    if gpu:
        print("Using GPU acceleration")
        # os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
        os.environ["CUDA_VISIBLE_DEVICES"] = f"{CUDA_VISIBLE_DEVICES}"  # feasible
        os.system("echo $CUDA_VISIBLE_DEVICES")
        # os.system("export CUDA_VISIBLE_DEVICES=1")
        # platform = openmm.Platform.getPlatformByName("CUDA")
        simulation = app.simulation.Simulation(pdb.topology, system, integrator,
                                               openmm.Platform.getPlatformByName("CUDA")
                                               ,{"DeviceIndex": f"{CUDA_VISIBLE_DEVICES}"})
    else:
        print("Using CPU")
        platform = openmm.Platform.getPlatformByName('CPU')
        simulation = app.simulation.Simulation(pdb.topology, system, integrator, platform, dict(Threads=str(1)))

    simulation.context.setPositions(pdb.positions)
    simulation.minimizeEnergy()
    # state = simulation.context.getState(getPositions=True)
    # pos2 = state.getPositions(asNumpy=True)
    # enforcePeriodicBox=False makes sure that the saved molecules are whole;
    # when openmm tries to keep the center of mass of molecule in the box, it will modify the x,y,z of every atoms separately;
    # which means only if x(or y or z) of COM is out of box, it will modify x of every atom so that x of COM is in the box;
    # Therefore, if just the x of a single atom is out of box, x might not be modified into the box;
    # and the center of simulation box is (L/2, L/2, L/2)
    simulation.reporters.append(app.dcdreporter.DCDReporter(f'{record}/production.dcd', wfreq, enforcePeriodicBox=False, append=False))

    simulation.reporters.append(
        app.statedatareporter.StateDataReporter(f'{record}/statedata_production.log', int(wfreq),
                                                step=True, speed=True, elapsedTime=True, separator='\t', progress=True,
                                                remainingTime=True, totalSteps=steps))
    simulation.reporters.append(
        app.checkpointreporter.CheckpointReporter(file=f"{record}/checkpoint_production.chk",
                                                  reportInterval=wfreq))


    starttime = time.time()  # begin timer
    assert steps%wfreq==0
    print(f"Total frames: {steps//wfreq}; the first 10 frames will be discarded")
    for _ in progress_bar(list(range(steps//wfreq))):
        simulation.step(wfreq)
    endtime = time.time()  # end timer
    target_seconds = endtime - starttime  # total used time
    print(
        f"{record} total simulations used time: {target_seconds // 3600}h {(target_seconds // 60) % 60}min {np.round(target_seconds % 60, 2)}s")

def fix_topology(dcd,pdb):
    """
    Changes atom names to CA; even if CG-strategy is COM, it is still CA for convenience
    """
    t = md.load_dcd(dcd,pdb)
    cgtop = md.Topology()
    cgchain = cgtop.add_chain()
    for atom in t.top.atoms:
        cgres = cgtop.add_residue(atom.name, cgchain)
        cgtop.add_atom('CA', element=md.element.carbon, residue=cgres)
    traj = md.Trajectory(t.xyz, cgtop, t.time, t.unitcell_lengths, t.unitcell_angles)
    traj = traj.superpose(traj, frame=0)
    return traj

def visualize_traj(record):
    traj_dcd = fix_topology(f'{record}/{record}.dcd', f'{record}/{record}.pdb')
    traj_dcd = traj_dcd.superpose(traj_dcd[0])
    traj_dcd[0].save_pdb(f'{record}/{record}_first.pdb')
    traj_dcd[-1].save_pdb(f'{record}/{record}_last.pdb')
    traj_dcd.save_trr(f"{record}/{record}.trr")
    traj_dcd.save_dcd(f"{record}/{record}_CA.dcd")

def centerDCD(config):
    record = config["record"]
    discard_first_nframes = config["discard_first_nframes"]
    initial_type = config["initial_type"]
    seq = config["seq"]
    residues = load_parameters(record, initial_type)
    top = md.Topology()
    chain = top.add_chain()
    for resname in seq:
        residue = top.add_residue(residues.loc[resname, 'three'], chain)
        top.add_atom(residues.loc[resname, 'three'], element=md.element.carbon, residue=residue)
    for i in range(len(seq) - 1):
        top.add_bond(top.atom(i), top.atom(i + 1))
    traj = md.load_dcd(f"{record}/production.dcd", top=top)[discard_first_nframes:]
    traj = traj.image_molecules(inplace=False, anchor_molecules=[set(traj.top.chain(0).atoms)], make_whole=True)
    traj.center_coordinates()
    traj.xyz += traj.unitcell_lengths[0, 0] / 2
    # print(f'Number of frames: {traj.n_frames}')
    traj.save_dcd(f'{record}/{record}.dcd')
    traj[0].save_pdb(f'{record}/{record}.pdb')
    visualize_traj(record)

def calRg(record, initial_type):
    df = load_parameters(record, initial_type).set_index("three")
    t = md.load_dcd(f"{record}/{record}.dcd", f"{record}/{record}.pdb")  # nm
    residues = [res.name for res in t.top.atoms]
    masses = df.loc[residues,'MW'].values
    masses[0] += 2
    masses[-1] += 16
    # calculate the center of mass
    cm = np.sum(t.xyz*masses[np.newaxis,:,np.newaxis],axis=1)/masses.sum()
    # calculate residue-cm distances
    si = np.linalg.norm(t.xyz - cm[:,np.newaxis,:],axis=2)
    # calculate rg
    rgarray = np.sqrt(np.sum(si**2*masses,axis=1)/masses.sum())
    np.save(f"{record}/Rg_traj.npy", rgarray)  # this contains Rg trajectory (n_frames,)
    print(f"Rg: {np.mean(rgarray)} nm")


config = yaml.safe_load(open(f"{name}/config_sim.yaml", 'r'))
simulate(config, "0")
centerDCD(config)

In [ ]:
# @title 6. Analysis
calRg(config["record"], config["initial_type"])

In [ ]:
#@title (Optional) Reconstruct all-atom trajectory
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import yaml
config = yaml.safe_load(open(f"{name}/config_sim.yaml", 'r'))
record = config["record"]
isIDP = config["isIDP"]
cg_model = "CalphaBasedModel" if isIDP else "ResidueBasedModel"
print(f"using {cg_model}")
!convert_cg2all -p {record}/{record}_first.pdb -d {record}/{record}.dcd -o {record}/{record}_AA.dcd -opdb {record}/{record}_topAA.pdb --cg {cg_model}

In [ ]:
# @title 7. Download results

# @markdown In this zip file:

# @markdown `$protein_name.dcd`: the coarse-grained trajectory file;

# @markdown `$protein_name.pdb`: the coarse-grained topology file;

# @markdown After reconstruction (Optional)

# @markdown `$protein_name_AA.dcd`: the reconstructed trajectory file;

# @markdown `$protein_name_topAA.pdb`: the all-atom topology file;
import subprocess
zipper = f'zip -r {name}.zip {name}'
subprocess.run(zipper.split())
files.download(f'{name}.zip')